<a href="https://colab.research.google.com/github/kylo-dev/gachon-machinelearning/blob/main/5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 교차 검증과 그리드 서치

## 검증 세트

단점 : 검정 세트로 인해 훈련 세트가 줄어들음

In [1]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [2]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [4]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [5]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [6]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


## 결정 트리 최적화

In [9]:
best_score = 0
best_depth = None

for depth in range(1, 11):  # 트리의 최대 깊이를 1부터 10까지 시도
    dt = DecisionTreeClassifier(max_depth=depth, random_state=42)
    dt.fit(sub_input, sub_target)

    # 검증 데이터로 정확도 평가
    val_score = dt.score(val_input, val_target)

    # 현재 최고 점수보다 높으면 최적 모델로 선택
    if val_score > best_score:
        best_score = val_score
        best_depth = depth

# 최적 모델로 훈련 및 테스트 데이터에 대한 정확도 평가
best_model = DecisionTreeClassifier(max_depth=best_depth, random_state=42)
best_model.fit(sub_input, sub_target)

train_accuracy = best_model.score(sub_input, sub_target)
test_accuracy = best_model.score(test_input, test_target)

print("Best Depth:", best_depth)
print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)

Best Depth: 8
Training Accuracy: 0.9018522973298051
Test Accuracy: 0.8576923076923076


## 교차 검증

검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복함

In [7]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.0236187 , 0.01151943, 0.0115757 , 0.01146984, 0.02025437]), 'score_time': array([0.00224686, 0.00203252, 0.00224066, 0.00208569, 0.00228548]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [8]:
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


### Stratified K-Fold 교차 검증

이 방법은 클래스 분포가 불균형한 경우에 유용하며, 각 하위 집합(폴드) 내에서 클래스 비율이 유지되도록 데이터를 나눕니다.

* n_splits는 폴드(fold)의 개수를 나타냅니다. 여기서는 10개의 폴드를 사용합니다.
* shuffle=True는 데이터를 무작위로 섞는 옵션입니다.
* random_state=42는 난수 발생을 위한 시드(seed)를 설정합니다

In [10]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.8537647145924335


In [11]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.856841929746554


## 하이퍼파라미터 튜닝

하이퍼파라미터란, 모델이 학습할 수 없어서 사용자가 지정해야만 하는 파라미터

In [12]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [13]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=1)

gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [15]:
dt = gs.best_estimator_ # 최적의 모델
print(dt.score(train_input, train_target))

0.9615162593804117


In [16]:
print(gs.best_params_) # 최적의 파라미터

{'min_impurity_decrease': 0.0001}


In [17]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [18]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


### 두 번째 하이퍼 튜닝

In [19]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [20]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [21]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


### 교차 검증과 검증 데이터

**교차 검증 (Cross-Validation):**

교차 검증은 주어진 데이터를 훈련 데이터와 검증 데이터로 여러 번 나누어서 모델을 평가하는 기법입니다.

주요 목적은 모델의 일반화 성능을 더 정확하게 평가하고, 과적합(overfitting)을 방지하는 것입니다.

교차 검증에서 검증 데이터는 훈련 데이터의 일부 중 하나로 사용됩니다.

</br>

**검증 데이터 (Validation Data):**

검증 데이터는 모델 학습 및 하이퍼파라미터 튜닝 단계에서 주로 사용됩니다.

주요 목적은 모델의 하이퍼파라미터를 조정하고 최적 모델을 선택하기 위한 것입니다.

검증 데이터는 훈련 데이터와 분리하여 사용되며, 모델을 여러 번 훈련하고 평가하여 하이퍼파라미터를 조정합니다.

최종 모델을 선택한 후, 테스트 데이터를 사용하여 모델의 일반화 성능을 최종적으로 평가합니다.

1. 주어진 데이터를 훈련 데이터와 테스트 데이터로 분리합니다.
2. 훈련 데이터를 사용하여 모델을 훈련하고, 검증 데이터를 사용하여 하이퍼파라미터를 조정합니다.
3. 최적의 모델을 선택하고, 테스트 데이터를 사용하여 모델의 일반화 성능을 평가합니다.


### 랜덤 서치

In [22]:
from scipy.stats import uniform, randint

In [23]:
rgen = randint(0, 10)
rgen.rvs(10)

array([7, 7, 4, 2, 0, 8, 8, 0, 8, 6])

In [24]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([113, 107,  88,  94,  95,  98,  95, 102, 102, 106]))

In [25]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.05159713, 0.12548515, 0.41189731, 0.01501398, 0.83958147,
       0.28497101, 0.67337704, 0.76448428, 0.77597277, 0.96762737])

In [26]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [27]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ff718b8be20>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7ff718b8bd00>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ff718b89420>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ff718b8b6d0>},
                   random_state=42)

In [28]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [29]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [30]:
dt = gs.best_estimator_

print(dt.score(test_input, test_target))

0.86


## 확인문제

In [32]:
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ff718b8be20>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7ff718b8bd00>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ff718b89420>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ff718b8b6d0>},
                   random_state=42)

In [33]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077


### 세가지 특성 중 하나로만 그리드서치

In [34]:
sugar = wine['sugar'].to_numpy()
sugar = sugar.reshape(-1,1)
target = wine['class'].to_numpy()

train_sugar, test_sugar,train_sugar_target, test_sugar_target = train_test_split(
    sugar, target, test_size=0.2, random_state=42, stratify=target)

In [35]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [36]:
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_sugar, train_sugar_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ff718b8b8b0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7ff718b8b7f0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ff718c7be80>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ff718bd7d00>},
                   random_state=42)

In [37]:
print(gs.best_params_)

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}


In [38]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8362499074553934


In [39]:
dt = gs.best_estimator_

print(dt.score(test_sugar, test_sugar_target))

0.8392307692307692
